# Experiment 005: Per-Target Heterogeneous Model WITHOUT TTA

Based on Loop 4 analysis:
- **TTA HURTS performance!** RF with TTA: 0.0932 vs RF without TTA: 0.0716 (30% worse with TTA!)
- Per-target (HGB+ETR) WITHOUT TTA achieves **0.0603 MAE on full data** (33% improvement!)
- Expected combined score: **0.0623** (22.7% better than current 0.0805)

**CRITICAL CHANGES from exp_004:**
1. REMOVE TTA completely - no data augmentation, no test-time averaging
2. Use same per-target approach for BOTH single and mixed solvents
3. Keep combined features (0.8 acs_pca + 0.2 spange)

**TEMPLATE COMPLIANCE**: Last 3 cells are EXACTLY as template, NO cells after them.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = '/home/data'
torch.set_default_dtype(torch.double)
print("Setup complete")

Setup complete


In [2]:
# --- UTILITY FUNCTIONS ---
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load both feature sets
SPANGE_DF = load_features('spange_descriptors')
ACS_PCA_DF = load_features('acs_pca_descriptors')
print(f"Spange: {SPANGE_DF.shape}, ACS_PCA: {ACS_PCA_DF.shape}")

Spange: (26, 13), ACS_PCA: (24, 5)


In [3]:
# --- BASE CLASSES ---
class SmilesFeaturizer(ABC):
    def __init__(self): raise NotImplementedError
    def featurize(self, X): raise NotImplementedError

class BaseModel(ABC):
    def __init__(self): pass
    def train_model(self, X_train, y_train): raise NotImplementedError
    def predict(self): raise NotImplementedError

In [4]:
# --- HYBRID PER-TARGET MODEL WITHOUT TTA ---
class HybridPerTargetModel(BaseModel):
    """Per-target heterogeneous model WITHOUT TTA.
    
    Key insight from Loop 4 analysis:
    - TTA HURTS performance on mixed solvents (30% worse!)
    - Removing TTA achieves 0.0603 MAE on full data (33% improvement)
    
    Architecture:
    - SM: HistGradientBoostingRegressor (max_depth=7, max_iter=700, lr=0.04)
    - Products: ExtraTreesRegressor (n_estimators=500, max_depth=10, min_samples_leaf=2)
    - Feature weighting: 0.8 acs_pca + 0.2 spange
    - NO TTA - no data augmentation, no test-time averaging
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.mixed = (data == 'full')
        self.targets = ['Product 2', 'Product 3', 'SM']
        
        # Load both feature sets
        self.spange = SPANGE_DF
        self.acs_pca = ACS_PCA_DF
        
        # Scalers for each feature set
        self.scaler_spange = StandardScaler()
        self.scaler_acs = StandardScaler()
        
        # Models: {target: {feature_set: model}}
        self.models = {}
        
        # Feature weights: 0.8 acs_pca + 0.2 spange
        self.acs_weight = 0.8
        self.spange_weight = 0.2

    def _build_features(self, X, feature_df):
        """Build features with Arrhenius kinetics - NO augmentation."""
        rt = X['Residence Time'].values.astype(np.float64).reshape(-1, 1)
        temp = X['Temperature'].values.astype(np.float64).reshape(-1, 1)
        
        # Arrhenius kinetic features
        temp_k = temp + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(rt + 1e-6)
        interaction = inv_temp * log_time
        
        if self.mixed:
            pct = X['SolventB%'].values.reshape(-1, 1)
            A = feature_df.loc[X['SOLVENT A NAME']].values
            B = feature_df.loc[X['SOLVENT B NAME']].values
            solvent_feats = A * (1 - pct) + B * pct
            return np.hstack([rt, temp, inv_temp, log_time, interaction, pct, solvent_feats])
        else:
            solvent_feats = feature_df.loc[X['SOLVENT NAME']].values
            return np.hstack([rt, temp, inv_temp, log_time, interaction, solvent_feats])

    def train_model(self, X_train, y_train):
        # Build features - NO AUGMENTATION!
        X_spange = self._build_features(X_train, self.spange)
        X_acs = self._build_features(X_train, self.acs_pca)
        
        # Scale
        X_spange_sc = self.scaler_spange.fit_transform(X_spange)
        X_acs_sc = self.scaler_acs.fit_transform(X_acs)
        
        y = y_train.values
        
        # Train per-target models
        for i, target in enumerate(self.targets):
            y_target = y[:, i]
            
            if target == 'SM':
                # HistGradientBoosting for SM
                model_spange = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04, random_state=42
                )
                model_acs = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04, random_state=42
                )
            else:
                # ExtraTrees for Products
                model_spange = ExtraTreesRegressor(
                    n_estimators=500, max_depth=10, min_samples_leaf=2,
                    random_state=42, n_jobs=-1
                )
                model_acs = ExtraTreesRegressor(
                    n_estimators=500, max_depth=10, min_samples_leaf=2,
                    random_state=42, n_jobs=-1
                )
            
            model_spange.fit(X_spange_sc, y_target)
            model_acs.fit(X_acs_sc, y_target)
            
            self.models[target] = {'spange': model_spange, 'acs': model_acs}

    def predict(self, X):
        # Build features - NO TTA!
        X_spange = self._build_features(X, self.spange)
        X_acs = self._build_features(X, self.acs_pca)
        
        X_spange_sc = self.scaler_spange.transform(X_spange)
        X_acs_sc = self.scaler_acs.transform(X_acs)
        
        preds_all = []
        for target in self.targets:
            p_spange = self.models[target]['spange'].predict(X_spange_sc)
            p_acs = self.models[target]['acs'].predict(X_acs_sc)
            # Weighted combination: 0.8 acs + 0.2 spange
            p_combined = self.acs_weight * p_acs + self.spange_weight * p_spange
            preds_all.append(p_combined.reshape(-1, 1))
        
        preds = np.hstack(preds_all)
        preds = np.clip(preds, 0, 1)
        return torch.tensor(preds, dtype=torch.double)

In [5]:
# --- QUICK VALIDATION TEST ---
print("Testing HybridPerTargetModel (NO TTA)...")
X_test, Y_test = load_data("single_solvent")

# Quick leave-one-out test on first 5 solvents
errors = []
split_gen = generate_leave_one_out_splits(X_test, Y_test)
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(split_gen):
    if i >= 5: break
    model = HybridPerTargetModel(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors.append(mae)
    solvent = test_X['SOLVENT NAME'].iloc[0]
    print(f"Fold {i} ({solvent}): MAE = {mae:.4f}")

print(f"\nQuick test MAE (single): {np.mean(errors):.4f}")

# Also test on full data
print("\nTesting on full data (NO TTA)...")
X_full, Y_full = load_data("full")
errors_full = []
split_gen = generate_leave_one_ramp_out_splits(X_full, Y_full)
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(split_gen):
    if i >= 3: break
    model = HybridPerTargetModel(data='full')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors_full.append(mae)
    print(f"Fold {i}: MAE = {mae:.4f}")

print(f"\nQuick test MAE (full): {np.mean(errors_full):.4f}")

Testing HybridPerTargetModel (NO TTA)...


Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol): MAE = 0.1446


Fold 1 (2,2,2-Trifluoroethanol): MAE = 0.0962


Fold 2 (2-Methyltetrahydrofuran [2-MeTHF]): MAE = 0.0369


Fold 3 (Acetonitrile): MAE = 0.0813


Fold 4 (Acetonitrile.Acetic Acid): MAE = 0.1077

Quick test MAE (single): 0.0934

Testing on full data (NO TTA)...


Fold 0: MAE = 0.0547


Fold 1: MAE = 0.0885


Fold 2: MAE = 0.0598

Quick test MAE (full): 0.0677


## Template-Compliant Cross-Validation

The following 3 cells are the FINAL 3 cells - EXACTLY as in the template.

In [6]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = HybridPerTargetModel(data='single') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:01,  1.87s/it]

2it [00:03,  1.92s/it]

3it [00:05,  1.91s/it]

4it [00:07,  1.89s/it]

...

21it [00:40,  1.90s/it]

22it [00:42,  1.92s/it]

23it [00:44,  1.93s/it]

24it [00:46,  1.93s/it]

24it [00:46,  1.92s/it]

In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = HybridPerTargetModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:02,  2.06s/it]

2it [00:04,  2.00s/it]

3it [00:06,  2.00s/it]

4it [00:08,  2.03s/it]

...

10it [00:20,  2.04s/it]

11it [00:22,  2.04s/it]

12it [00:24,  2.04s/it]

13it [00:26,  2.04s/it]

13it [00:26,  2.04s/it]

In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################